# **Metpy Station Plots: Southern California Edison Stations**
## This notebook performs the following task(s):
> - #### Creates a cartopy map and plots meteoroloigcal variables from individual Southern California Edison (SCE) weather stations using the metpy "StationPlot" function.

## **Import packages**
#### Links to documentation for packages
> - #### [pathlib](https://docs.python.org/3/library/pathlib.html) | [numpy](https://numpy.org/doc/1.21/) | [xarray](https://docs.xarray.dev/en/stable/) | [pandas](https://pandas.pydata.org/pandas-docs/version/1.3.5/) | [cartopy](https://scitools.org.uk/cartopy/docs/latest/) | [matplotlib](https://matplotlib.org/3.5.3/index.html) | [metpy](https://unidata.github.io/MetPy/latest/index.html)
> - #### Documentation for packages linked above should mostly correspond to the most stable versions, which may not be the exact versions used when creating this notebook.
> - #### Comments are also included in the actual code cells. Some comments contain links that point to places where I copied or adapted code to fit my needs. Although I tried to these include links for all instancs of copying, it is possible that there may code snippets that I did not do this for.

In [ ]:
#-----------------------------------------------------
#Entire packages
import pathlib
import numpy as np
import xarray as xr
import pandas as pd

#cartopy imports
import cartopy.crs as ccrs

#matplotlib imports
import matplotlib.pyplot as plt
import matplotlib.patheffects as path_effects
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

#metpy imports
from metpy.units import units
from metpy.plots import StationPlot, StationPlotLayout
from metpy.calc import dewpoint_from_relative_humidity

#function notebook
%run functions_swex_iop_10.ipynb
#-----------------------------------------------------

## **Define variables that point to relative paths for relevant data**
#### Data Information
> - #### **Southern California Edison (SCE) Station Timeseries Data**
>> - #### Time Resolution: 10-minutes
>> - #### SCE data downloaded from the [Synoptic data website](https://synopticdata.com/) using the [SynopticPy package written by Brian Blaylock](https://github.com/blaylockbk/SynopticPy). 

In [ ]:
#-----------------------------------------------------
#Define a list of stations that we want to grab from the data directory
station_id_str_list = ['351SE', 'SE492', 'SE745', 'SE505', 'SE750', 
                       'SE507', '421SE', 'SE234', 'SE504', 'SE741', 
                       'SE737', 'SE738', 'SE760', 'SE516', 'SE765', 
                       'SE053']

#Create empty list to store files we want to read in
station_files = []

#Iterate through each station ID, grab the file using the pathlib glob, and then add it to our empty list
for station_id in station_id_str_list:
    station_files.extend(pathlib.Path('./SWEX2022_datasets/SCE/').glob(f'*{station_id}*'))

#Display one file path to ensure we have files
display(len(station_files))
#-----------------------------------------------------

## **SCE variable grabbing, time reindexing, and dewpoint computation**
#### Notes
> - #### This cell grabs the timeseries values for a single SCE station variable for all of the SCE stations that we have. 
> - #### It also reindexes each variable from each station to all timeseries are the same length for all variables. All missing values are filled with NaNs.
> - #### We utilize metpy's ["dewpoint_from_relative_humidity"](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.dewpoint_from_relative_humidity.html) function to compute the station dewpoint for all stations using temperature and relative humidity.

In [ ]:
#-----------------------------------------------------
#Define storage locations for variables of interest
lon_list    = []
lat_list    = []
time_list   = []
u_wind_list = []
v_wind_list = []
temp_list   = []
relh_list   = []
dewp_list   = []
#-----------------------------------------------------
#For every single station file we have, do the following
for file_index, file in enumerate(station_files):
    
    #Open the file using xarray
    ds = xr.open_dataset(file)
    
    #Grab timestamps from each station
    time_list.append(ds['date_time'])
#-----------------------------------------------------
#Grab all the unique timestamps from all stations
#This will be used to reindex all the station timeseries for all variables so they are all the same length
#Thanks to ChatGPT for helping with this code: https://chatgpt.com/share/6706d004-5d94-800f-9457-e22523c27c49
unique_times = np.sort(np.unique(np.concatenate(time_list)))
#-----------------------------------------------------
#For every single station file we have, do the following
for file_index, file in enumerate(station_files):
    
    #Open the file using xarray
    ds = xr.open_dataset(file)
    
    #Grab station lon/lat
    lon_list.append(ds.longitude)
    lat_list.append(ds.latitude)
        
    #Grab U and V wind components (units: m/s)
    u_wind_list.append(ds['wind_u_set_1'].assign_attrs(ds.attrs).reindex(date_time=unique_times, fill_value=np.nan))
    v_wind_list.append(ds['wind_v_set_1'].assign_attrs(ds.attrs).reindex(date_time=unique_times, fill_value=np.nan))

    #Grab air temperature (units: °C)
    temp_list.append(ds['air_temp_set_1'].assign_attrs(ds.attrs).reindex(date_time=unique_times, fill_value=np.nan))
    
    #Grab relative humidity (units: %)
    relh_list.append(ds['relative_humidity_set_1'].assign_attrs(ds.attrs).reindex(date_time=unique_times, fill_value=np.nan))
    
    #Compute dewpoint using metpy function "dewpoint_from_relative_humidity".
    #We assign Pint units of °C assigned to temperature, and convert relative humidity units from % to a value between 0 and 1
    single_station_dewp = dewpoint_from_relative_humidity(ds['air_temp_set_1']*units.degC, ds['relative_humidity_set_1']/100)
    
    #Add a name to the dewpoint DataArray from each station to keep us sane
    single_station_dewp = single_station_dewp.rename(f'Metpy Derived Dewpoint: {ds.STID}').assign_attrs(ds.attrs).reindex(date_time=unique_times, fill_value=np.nan)
    
    #Append the computed 2m dewpoint pint.Quantity (array) into a list
    dewp_list.append(single_station_dewp)
#-----------------------------------------------------

## **Subset station timeseries for plotting and check length of each station subset timeseries**
#### Notes
> - #### See inline comments for details.

In [ ]:
#-----------------------------------------------------
#Define a list of plotting time strings that we will using for indexing the data
plotting_times_list = ['2022-05-12 17:00', '2022-05-12 21:00', '2022-05-12 01:00',
                       '2022-05-13 05:00', '2022-05-13 09:00', '2022-05-13 13:00']

#Subset station data using list comprehension
u_wind_subset_list = [u_wind_single.sel(date_time=plotting_times_list) for u_wind_single in u_wind_list]
v_wind_subset_list = [v_wind_single.sel(date_time=plotting_times_list) for v_wind_single in v_wind_list]
temp_subset_list   = [temp_single.sel(date_time=plotting_times_list) for temp_single in temp_list]
dewp_subset_list   = [dewp_single.sel(date_time=plotting_times_list) for dewp_single in dewp_list]

#Data Check: Make sure all arrays for each variable are the same length
#ChatGPT helped write this check: https://chat.openai.com/share/4836a13a-ca54-42f8-937e-a2596ee24460

#Define a list which contains all the lists we want to check
lists_to_check = [u_wind_subset_list, v_wind_subset_list, temp_subset_list, dewp_subset_list]

#For each list, do the following:
for list_index, list_to_check in enumerate(lists_to_check):
    
    #Compute array lengths for all arrays in a single variable list
    array_lengths_in_list = [array.shape[0] for array in list_to_check]
    
    #If all arrays lenghts equal the first length, print that all arrays are equal in length.
    if all(length == array_lengths_in_list[0] for length in array_lengths_in_list) == True:
        print(f'All 1D arrays are equal in length')
    #Else print that all arrays are not equal in length.
    else:
        print(f'All 1D arrays are NOT equal in length.')
#-----------------------------------------------------

## **Iteratively plot station variables on a metpy station plot for all stations**
#### Notes
> - #### Great resources for creating metpy station plot: [Basic Station Plot](https://unidata.github.io/MetPy/latest/examples/plots/Station_Plot.html) | [Station Plot with Custom Layout](https://unidata.github.io/MetPy/latest/examples/plots/Station_Plot_with_Layout.html)
> - #### Variables plotted:
>> - #### Wind speed and direction (barb; units: m/s converted to knots)
>> - #### Air Temperature (red value; units: °C)
>> - #### Derived Dewpoint (green value; units: °C; derived from using air temperature, relative humidity, and the metpy function ["dewpoint_from_relative_humidity"](https://unidata.github.io/MetPy/latest/api/generated/metpy.calc.dewpoint_from_relative_humidity.html))

In [ ]:
#-----------------------------------------------------
#CARTOPY MAP SETUP

#Define font dictionaries
fontdict_text_annotation = {'fontsize': 24, 'fontweight': 'normal', 'fontname': 'Nimbus Roman'}
fontdict_title_labels    = {'fontsize': 24, 'fontweight': 'bold', 'fontname': 'Nimbus Roman'}

#Create a list of strings to mark subplot windows with sequential letters
subplot_letter_list = ['a)', 'b)', 'c)', 'd)', 'e)', 'f)']

#Define plot and data coordinate reference system
plot_crs = ccrs.PlateCarree()
data_crs = ccrs.PlateCarree()

#Define cartopy plot domain in lat/lon coordinates
lon_lat_tick_num = [4, 4]
lon_lat_extent   = [-119.75, -119.58, 34.38, 34.48]
lon_lat_ticks    = [-119.75, -119.58, 34.38, 34.48]

#Define a a list of datetimes that we want to plot
time_list = pd.to_datetime(['2022-05-12 17:00', '2022-05-12 21:00', '2022-05-13 01:00', '2022-05-13 05:00', '2022-05-13 09:00', '2022-05-13 13:00'])

#Draw cartopy basemap and return fig/axis
fig, axs = cartopy_basemap_subplots(plot_crs=plot_crs, data_crs=data_crs, 
                                    nrows=3, ncols=2, fig_size=(20,20), 
                                    wspace_float=None, hspace_float=None,
                                    lon_lat_extent=lon_lat_extent, 
                                    lon_lat_ticks=lon_lat_ticks, 
                                    lon_lat_tick_num=lon_lat_tick_num, 
                                    lon_lat_ticks_on=True, 
                                    xtick_ytick_set_list=[False, False, False, False, True, False],
                                    high_res_coastline=True, 
                                    high_res_wrf_topo_sb_bool=True, 
                                    low_res_wrf_topo_sb_bool=False, 
                                    low_res_wrf_topo_ca_bool=False, 
                                    wrf_topo_colorbar_each_plot_bool=False, 
                                    wrf_topo_colorbar_entire_figure_bool=True,
                                    scale_bar_bool=False, scale_bar_position=None, 
                                    inset_ca_bool=False, inset_bbox_position=None, 
                                    ocean_color='lightskyblue')
#-----------------------------------------------------
#For every timestep we have, do the following
for ax_index, (ax, time_stamp) in enumerate(zip(axs, time_list)):

    #Add point that shows station locations
    ax.scatter(lon_list, lat_list, s=100,  marker='o', color='brown', edgecolor='black', transform=data_crs, zorder=10)

    #Add some points for a few locations of interest
    ax.scatter(-119.70871, 34.45508,   s=100,  marker='s', color='black', edgecolor='black', transform=data_crs, zorder=10)
    ax.scatter(-119.659404, 34.440288, s=100,  marker='s', color='black', edgecolor='black', transform=data_crs, zorder=10)
    
    #Define a list of tuples (label, lon, lat) for plotting on map
    #('Isla Vista',-119.863440,34.412815)
    labels = [('BOT',-119.708,34.46), ('MFS2',-119.660,34.445),
              ('Montecito',-119.65,34.415), ('Santa\nBarbara',-119.710209, 34.413321), 
              ('Santa Ynez Mountains',-119.63,34.47), ('Pacific Ocean',-119.65,34.395)]

    #For each label we have annotate that point on the map with the label name
    for label_index, (label, lon, lat) in enumerate(labels):

        #Add in label text
        label_text = ax.annotate(label, xy=(lon,lat), xytext=((0,0)), color='black', textcoords='offset points', horizontalalignment='center', verticalalignment='center', **fontdict_text_annotation)
        
        #Add text effect to label
        #https://matplotlib.org/3.2.1/tutorials/advanced/patheffects_guide.html
        label_text.set_path_effects([path_effects.Stroke(linewidth=4, foreground='white'), path_effects.Normal()])
       
    #Add title with time/date information
    datetime_utc     = time_stamp
    datetime_pdt     = datetime_utc-pd.Timedelta(7, unit='h')
    datetime_pdt_str = f'{str(datetime_pdt.hour).zfill(2)}00 PDT {str(datetime_pdt.day).zfill(2)} May 2022'
    ax.set_title(f'{subplot_letter_list[ax_index]} {datetime_pdt_str}', **fontdict_title_labels)
    
    #Base for station plot
    station_plot = StationPlot(ax=ax, x=lon_list, y=lat_list, fontsize=14, transform=data_crs)

    #Station wind barb (converting from m/s to knots)
    #We create a list of u and v wind speed xarray DataArrays which are converted from m/s to knots and also indexed to grab only the timesteps near the beginning of the hour for all 18 stations
    station_barbs = station_plot.plot_barb(u=[u_wind_single_station_single_time[ax_index]*1.94384 for u_wind_single_station_single_time in u_wind_subset_list], 
                                           v=[v_wind_single_station_single_time[ax_index]*1.94384 for v_wind_single_station_single_time in v_wind_subset_list], 
                                           sizes=dict(emptybarb=0), rounding=True, linewidth=1.5, zorder=10)
    #Station temperature
    station_temps = station_plot.plot_parameter(location='NE', parameter=[temp_single_station_single_time[ax_index].values for temp_single_station_single_time in temp_subset_list], formatter='.1f', color='red', weight='semibold', zorder=10)

    #Station dewpoint (computed)
    station_dews  = station_plot.plot_parameter(location='SE', parameter=[dewp_single_station_single_time[ax_index].values for dewp_single_station_single_time in dewp_subset_list], formatter='.1f', color='darkgreen', weight='semibold', zorder=10)
#-----------------------------------------------------    
#Save figure
plt.savefig(f'./figures/figure_05_observations_sce.png', bbox_inches='tight', dpi=500)                                                             
#-----------------------------------------------------